In [ ]:
!pip install langchain langchain-openai langchain-community langchain-cohere langchain-mistralai faiss-cpu rank-bm25 -q

In [3]:
import os
os.environ['COHERE_API_KEY'] = 'WbDwiNgZambTJnI3Uwq8nr5LZ5P5AJn8lPWjWnrI'

In [5]:
from typing import List
from langchain_community.document_loaders import TextLoader, DirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_mistralai import MistralAIEmbeddings
from langchain_cohere import CohereEmbeddings
from langchain_community.vectorstores import FAISS
from langchain_community.retrievers import BM25Retriever
from langchain.retrievers import EnsembleRetriever
from langchain.retrievers.document_compressors import LLMChainExtractor
from langchain.retrievers import ContextualCompressionRetriever
from langchain_core.documents import Document

class OptimizedRAGPipeline:
    def __init__(self,
                 docs_path: str = 'doc/',
                 chunk_size: int = 2000,
                 chunk_overlap: int = 200,
                 embedding_model: str = 'embed-english-v3.0',
                 use_reranker: bool = True,
                 use_compression: bool = True):
        """
        Initialize the optimized RAG pipeline

        Args:
            docs_path (str): Path to the documents directory
            chunk_size (int): Size of document chunks
            chunk_overlap (int): Overlap between chunks
            embedding_model (str): Cohere embedding model name (note: uses 'model' parameter in newer versions)
            use_reranker (bool): Whether to use reranking
            use_compression (bool): Whether to use contextual compression
        """
        self.docs_path = docs_path
        self.chunk_size = chunk_size
        self.chunk_overlap = chunk_overlap
        self.embedding_model = embedding_model
        self.use_reranker = use_reranker
        self.use_compression = use_compression

        # Initialize components
        self.documents = self._load_documents()
        self.embeddings = self._initialize_embeddings()
        self.vector_store = self._create_vector_store()
        self.retriever = self._build_retriever()

    def _load_documents(self) -> List[Document]:
        """Load and split documents from the specified path"""

        try:
            # Load documents from directory
            loader = DirectoryLoader(
                self.docs_path,
                glob='*.txt',
                loader_cls=TextLoader,
                loader_kwargs={"encoding": "utf-8"}
            )

            # Split documents with optimized chunking strategy
            text_splitter = RecursiveCharacterTextSplitter(
                chunk_size=self.chunk_size,
                chunk_overlap=self.chunk_overlap,
                separators=["\n\n", "\n", ". ", " ", ""],
                length_function=len
            )

            documents = loader.load_and_split(text_splitter)

            return documents

        except Exception as e:

            return []

    def _initialize_embeddings(self):
        """Initialize the embedding model"""

        try:
            return CohereEmbeddings(model=self.embedding_model)
        except Exception as e:
            try:
                return CohereEmbeddings(model=self.embedding_model)
            except Exception as fallback_error:
                return CohereEmbeddings(model=self.embedding_model)

    def _create_vector_store(self):
        """Create a vector store from documents and embeddings"""

        try:
            return FAISS.from_documents(self.documents, self.embeddings)
        except Exception as e:

            return None

    def _build_retriever(self, k: int = 3):
        """Build an optimized retriever with multiple strategies"""

        # Create base vector retriever
        vector_retriever = self.vector_store.as_retriever(search_kwargs={"k": k})

        # Create BM25 retriever for keyword-based retrieval
        bm25_retriever = BM25Retriever.from_documents(self.documents)
        bm25_retriever.k = k

        # Combine retrievers with weights
        ensemble_retriever = EnsembleRetriever(
            retrievers=[vector_retriever, bm25_retriever],
            weights=[0.7, 0.3]
        )

        # Add contextual compression if enabled
        if self.use_compression:
            try:
                try:
                    from langchain_cohere import ChatCohere
                    llm = ChatCohere(
                        cohere_api_key=os.environ.get('COHERE_API_KEY'),
                        temperature=0
                    )
                except Exception:
                    print("Error in contextual compression")

                compressor = LLMChainExtractor.from_llm(llm)

                return ContextualCompressionRetriever(
                    base_compressor=compressor,
                    base_retriever=ensemble_retriever
                )
            except Exception as e:
                return ensemble_retriever
        else:
            return ensemble_retriever

    def retrieve(self, query: str, k: int = 3) -> List[Document]:
        """
        Retrieve relevant documents for a query

        Args:
            query (str): The search query
            k (int): Number of documents to retrieve

        Returns:
            List[Document]: List of retrieved documents
        """

        try:
            # Use the new invoke method instead of deprecated get_relevant_documents
            return self.retriever.invoke(query)
        except Exception as e:

            # Fallback to simple vector retrieval if ensemble fails
            try:
                return self.vector_store.similarity_search(query, k=k)
            except Exception as inner_e:
                return []

    def get_retrieval_content(self, query: str, k: int = 3) -> List[str]:
        """
        Get the content of retrieved documents for a query

        Args:
            query (str): The search query
            k (int): Number of documents to retrieve

        Returns:
            List[str]: List of document contents
        """
        docs = self.retrieve(query, k)
        return [doc.page_content if doc.page_content != 'NO_OUTPUT.' else '' for doc in docs]


# Initialize the RAG pipeline
rag_pipeline = OptimizedRAGPipeline()

In [6]:
retrieve = rag_pipeline.retrieve('How do I reset my password on GOKAP InnoTech')
retrieve

[Document(metadata={'source': 'doc\\faq.txt'}, page_content='### 1. "How do I reset my password?"\nTo reset your password on GOKAP InnoTech:\n1. Click on the "Login" button at the top right of the homepage\n2. Select "Forgot Password" below the login form\n3. Enter the email address associated with your account\n4. Check your email inbox for a password reset link (check spam/junk folders if not visible)\n5. Click the link and follow instructions to create a new password\n6. Use your new password to log in\n\nIf you don\'t receive the reset email within 10 minutes, you can contact our support team at support@gokapinnotech.com for assistance.'),
 Document(metadata={'source': 'doc\\main_corpus.txt'}, page_content='NO_OUTPUT.'),
 Document(metadata={'source': 'doc\\faq.txt'}, page_content='NO_OUTPUT.'),
 Document(metadata={'source': 'doc\\faq.txt'}, page_content='NO_OUTPUT.'),
 Document(metadata={'source': 'doc\\faq.txt'}, page_content='NO_OUTPUT.')]

In [7]:
retrieval_content = rag_pipeline.get_retrieval_content('How do I reset my password on GOKAP InnoTech')
retrieval_content

['### 1. "How do I reset my password?"\nTo reset your password on GOKAP InnoTech:\n1. Click on the "Login" button at the top right of the homepage\n2. Select "Forgot Password" below the login form\n3. Enter the email address associated with your account\n4. Check your email inbox for a password reset link (check spam/junk folders if not visible)\n5. Click the link and follow instructions to create a new password\n6. Use your new password to log in\n\nIf you don\'t receive the reset email within 10 minutes, you can contact our support team at support@gokapinnotech.com for assistance.',
 '',
 '',
 '',
 '']

In [8]:
from langchain_cohere import ChatCohere
llm = ChatCohere()

In [9]:
from langchain_core.prompts import ChatPromptTemplate

prompt = ChatPromptTemplate.from_template("""
  "You are a helpful customer support assistant for gokap innotech company. "
  "Your goal is to provide accurate, helpful, and concise responses based on the company's knowledge base. "
  Asked Question = {question}
  Retrived Context = {context}
""")

In [10]:
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough

retriever = rag_pipeline.get_retrieval_content

chain = (
    {"context" : retriever, "question" : RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)

In [11]:
answer = chain.invoke("How do I reset my password on GOKAP InnoTech")
answer

'To reset your password on GOKAP InnoTech, follow these steps:\n\n1. **Click on the "Login" button** at the top right of the homepage.  \n2. **Select "Forgot Password"** below the login form.  \n3. **Enter the email address** associated with your account.  \n4. **Check your email inbox** for a password reset link (also check spam/junk folders if it’s not visible).  \n5. **Click the link** and follow the instructions to create a new password.  \n6. **Use your new password** to log in.  \n\nIf you don’t receive the reset email within 10 minutes, please contact our support team at **support@gokapinnotech.com** for assistance. Let me know if you need further help!'

In [12]:
!pip install langchain-openai -qU

In [13]:
from langchain_openai import AzureChatOpenAI

In [14]:
endpoint = "https://ai-22bca102665700ai188063459630.openai.azure.com/"
model_name = "gpt-35-turbo-16k"
deployment = "gpt-35-turbo-16k"
subscription_key = "3rvBCoNuhTcQPmC7xgwiHY72VMcbzGxzwuWSMbWJsPcsIuJ8JtuDJQQJ99BDACHYHv6XJ3w3AAAAACOGuUwO"
api_version = "2024-12-01-preview"

In [15]:
evaluate_llm = AzureChatOpenAI(
    deployment_name=deployment,
    api_version= api_version,
    model='gpt-35-turbo-16k',
    azure_endpoint=endpoint,
    api_key=subscription_key
)

In [16]:
!pip install ragas -qU

In [ ]:
from ragas import SingleTurnSample, EvaluationDataset